<H1> Problem Solving with Search</H1>

# Some utility functions and imports

In [0]:
import pandas as pd
import json
from collections import deque
import heapq

from IPython.display import display, HTML
from functools import total_ordering

debug = True
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>")))

def format_search_log(log,limit=100):
  columns=['node to expand', 'path', 'left in frontier', 'valid actions','is_goal']
  return pretty_print(pd.DataFrame(log, columns=columns).head(limit))

def format_search_with_costs_log(log, limit=100):
  columns=['node to expand', 'path', 'g','f','left in frontier', 'valid actions','is_goal']
  return pretty_print(pd.DataFrame(log, columns=columns).head(limit))

# dict on python 3.7+ preserves insertion order.
# This is a quick way to create a set which preserves it as well.
# required for presentation purposes only.
def ordered_set(coll):
  return dict.fromkeys(coll).keys()

def swap_tuple(a_tuple, i, j):
  l = list(a_tuple)
  l[i],l[j] = l[j],l[i]
  return tuple(l)

def load_routes(routes, symmetric=True):
  def insert(frm,to,cost):
    if frm in G:
      G[frm][to]=cost
    else:
      G[frm] = {to:cost}
  G = {}
  routes = routes.splitlines()
  for route in routes:
    r = route.split(',')
    insert(r[0],r[1],int(r[2]))
    if symmetric:
      insert(r[1],r[0],int(r[2]))
  return G

In [0]:
#There is no priority in Python which allows custom sorting function
class PriorityQueue:

    def __init__(self, f=lambda x: x):
        self.heap = []
        self.f = f

    def append(self, item):
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        for item in items:
            self.append(item)

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        return len(self.heap)

    def __contains__(self, key):
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)
    def __repr__(self):
      return str(self.heap)

# Problem Formulation

## Sliding tile puzzle

In [0]:
class TilePuzzleProblem:
  
  def __init__(self, s_start, n):
    self.s_start = s_start
    self.n = n
    self.goal = tuple(range(0,n**2))
    self.moves = {
      'left': lambda x: x[0]>0,
      'right':lambda x: x[0]<n-1,
      'up':   lambda x: x[1]>0,
      'down': lambda x: x[1]<n-1        
    }
    self.transitions = {
      'left':  -1,
      'right': 1,
      'up':    -self.n,
      'down':  self.n        
    }
  
  def actions(self, s):
    blank = s.index(0)
    xy = (blank%self.n, blank//self.n)
    return [m for (m,f) in self.moves.items() if f(xy)]
  
  def succ(self, s, a):
    curr_i = s.index(0)
    new_i = curr_i + self.transitions[a]
    return swap_tuple(s, curr_i, new_i)
    
  def is_goal(self, s):
    return s==self.goal

  def step_cost(self, s, a):
    return 1

  def state_str(self, s):
    return '\n'.join( [str( s[i*self.n:(i+1)*self.n]) for i in range(0,self.n)])

In [0]:
s_start = (
    3, 1, 2,
    4, 0, 5,
    6, 7, 8
)


In [5]:
tp = TilePuzzleProblem(s_start,3)
print("Initial State:")
print(tp.state_str(s_start))
print(f'is_goal? {tp.is_goal(s_start)}')
print("Applicable actions for initial state:")
print(tp.actions(s_start))


print("\nGenerating a new state by moving the blank left")
s = tp.succ(s_start, 'left')
print(tp.state_str(s))
print(f'is_goal? {tp.is_goal(s)}')
print("Applicable actions for the new state:")
print(tp.actions(s))


print("\nGenerating a new state by moving up")
s = tp.succ(s, 'up')
print(tp.state_str(s))
print(f'is_goal? {tp.is_goal(s)}')


Initial State:
(3, 1, 2)
(4, 0, 5)
(6, 7, 8)
is_goal? False
Applicable actions for initial state:
['left', 'right', 'up', 'down']

Generating a new state by moving the blank left
(3, 1, 2)
(0, 4, 5)
(6, 7, 8)
is_goal? False
Applicable actions for the new state:
['right', 'up', 'down']

Generating a new state by moving up
(0, 1, 2)
(3, 4, 5)
(6, 7, 8)
is_goal? True


## Routing problem

In [0]:
class RoutingProblem:
 
  def __init__(self, s_start, goal, G):
    self.s_start = s_start
    self.goal = goal
    self.G = G  
  
  def actions(self, s):
    return self.G[s].keys()
  
  def succ(self, s, a):
    if a in self.G[s]:
      return a
    raise ValueError(f'No route from {s} to {a}')

  def is_goal(self, s):
    return s==self.goal

  def step_cost(self, s, a):
    return self.G[s][a]

  def state_str(self, s):
    return s
  
  def __repr__(self):
    return {'s_start':self.s_start, 'goal':self.goal, graph:'self.G'}

In [7]:
routes = """\
Ashkelon,Ashdod,35
Ashdod,Gan-Yavne,1
Ashdod,Yavne,11
Yavne,Gan-Yavne,13
Yavne,Tel-Aviv,25
Aluf Sade Interchange,Yavne,26
Yavne,Rehovot,6
Tel-Aviv,Aluf Sade Interchange,5
Rehovot,Aluf Sade Interchange,27
Aluf Sade Interchange,Bar-Ilan University,1
Bar-Ilan University,Kiryat Ono,4
"""
G = load_routes(routes, symmetric=True)
print(json.dumps(G, indent=4, sort_keys=True))
print(f"""The cost of traveling from Ashdod to Ashkelon is: {G['Ashdod']['Ashkelon']}""")

{
    "Aluf Sade Interchange": {
        "Bar-Ilan University": 1,
        "Rehovot": 27,
        "Tel-Aviv": 5,
        "Yavne": 26
    },
    "Ashdod": {
        "Ashkelon": 35,
        "Gan-Yavne": 1,
        "Yavne": 11
    },
    "Ashkelon": {
        "Ashdod": 35
    },
    "Bar-Ilan University": {
        "Aluf Sade Interchange": 1,
        "Kiryat Ono": 4
    },
    "Gan-Yavne": {
        "Ashdod": 1,
        "Yavne": 13
    },
    "Kiryat Ono": {
        "Bar-Ilan University": 4
    },
    "Rehovot": {
        "Aluf Sade Interchange": 27,
        "Yavne": 6
    },
    "Tel-Aviv": {
        "Aluf Sade Interchange": 5,
        "Yavne": 25
    },
    "Yavne": {
        "Aluf Sade Interchange": 26,
        "Ashdod": 11,
        "Gan-Yavne": 13,
        "Rehovot": 6,
        "Tel-Aviv": 25
    }
}
The cost of traveling from Ashdod to Ashkelon is: 35


In [8]:
s_start, goal = 'Yavne', 'Bar-Ilan University'

rp = RoutingProblem(s_start, goal, G)
print("Initial State:")
print(rp.state_str(rp.s_start))
print(f'is_goal? {rp.is_goal(s_start)}')
print("Applicable actions for initial state:")
print(rp.actions(s_start))


print("\nGenerating a new state by moving to Tel-Aviv")
s = rp.succ(s_start, 'Tel-Aviv')
print(rp.state_str(s))
print(f'is_goal? {rp.is_goal(s)}')
print("Applicable actions for the new state:")
print(rp.actions(s))


print("\nGenerating a new state by moving to Aluf-Sade Interchange")
s = rp.succ(s, 'Aluf Sade Interchange')
print(rp.state_str(s))
print(f'is_goal? {rp.is_goal(s)}')

print("\nGenerating a new state by moving to Bar-Ilan University")
s = rp.succ(s, 'Bar-Ilan University')
print(rp.state_str(s))
print(f'is_goal? {rp.is_goal(s)}')


Initial State:
Yavne
is_goal? False
Applicable actions for initial state:
dict_keys(['Ashdod', 'Gan-Yavne', 'Tel-Aviv', 'Aluf Sade Interchange', 'Rehovot'])

Generating a new state by moving to Tel-Aviv
Tel-Aviv
is_goal? False
Applicable actions for the new state:
dict_keys(['Yavne', 'Aluf Sade Interchange'])

Generating a new state by moving to Aluf-Sade Interchange
Aluf Sade Interchange
is_goal? False

Generating a new state by moving to Bar-Ilan University
Bar-Ilan University
is_goal? True


In [9]:
routes2 = """\
S,A,5
A,B,4
B,C,4
S,D,4
D,E,3
E,F,4
F,G,3
A,E,1
E,B,5
"""
G2 = load_routes(routes2, symmetric=True)
print(f"""The cost of traveling from A to B is: {G2['A']['B']}""")
G2

The cost of traveling from A to B is: 4


{'A': {'B': 4, 'E': 1, 'S': 5},
 'B': {'A': 4, 'C': 4, 'E': 5},
 'C': {'B': 4},
 'D': {'E': 3, 'S': 4},
 'E': {'A': 1, 'B': 5, 'D': 3, 'F': 4},
 'F': {'E': 4, 'G': 3},
 'G': {'F': 3},
 'S': {'A': 5, 'D': 4}}

# Definition of Node in a search graph

In [0]:
@total_ordering
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
      self.state = state
      self.parent = parent
      self.action = action
      self.path_cost = path_cost
      self.depth = 0
      if parent:
        self.depth = parent.depth + 1

    def expand(self, problem):
      return ordered_set([self.child_node(problem, action)
        for action in problem.actions(self.state)])

    def child_node(self, problem, action):
      next_state = problem.succ(self.state, action)
      next_node = Node(next_state, self, action,
                  self.path_cost+problem.step_cost(self.state, action))
      return next_node
    
    def solution(self):
      return [node.action for node in self.path()[1:]]

    def path(self):
      node, path_back = self, []
      while node:
          path_back.append(node)
          node = node.parent
      return list(reversed(path_back))
    def __repr__(self):
      return f"<{self.state}>"

    def __lt__(self, node):
      return self.state < node.state
    
    def __eq__(self, other):
      return isinstance(other, Node) and self.state == other.state

    def __ne__(self, other):
      return not (self == other)

    def __hash__(self):
        return hash(self.state)

# Uninformed Search algorithms

## Implementation

### BFS

In [0]:
def breadth_first_tree_search(problem):
  frontier = deque([Node(problem.s_start)])  # FIFO queue
  log = []
  while frontier:
    node = frontier.popleft()
    if debug:
      log.append((problem.state_str(node.state),node.solution(),len(frontier),problem.actions(node.state),problem.is_goal(node.state)))
    if problem.is_goal(node.state):
      return node.solution(), log
    frontier.extend(node.expand(problem))
  return None, log

def breadth_first_graph_search(problem):
  frontier = deque([Node(problem.s_start)])  # FIFO queue
  closed_list = set()
  log = []
  while frontier:
    node = frontier.popleft()
    if debug:
      log.append((problem.state_str(node.state),node.solution(),len(frontier),problem.actions(node.state),problem.is_goal(node.state)))
    if problem.is_goal(node.state):
        return node.solution(), log
    closed_list.add(node.state)
    for child in node.expand(problem):
        if child.state not in closed_list and child not in frontier:
            frontier.append(child)
  return None, log

### DFS

In [0]:
def depth_first_tree_search(problem):
    frontier = [(Node(problem.s_start))]  # Stack
    log = []
    while frontier:
        node = frontier.pop()
        if debug:
          log.append((problem.state_str(node.state),node.solution(),len(frontier),problem.actions(node.state),problem.is_goal(node.state)))
        if problem.is_goal(node.state) or (debug and len(log)>100):
          return node.solution(), log
        frontier.extend(node.expand(problem))
    return None, log

def depth_first_graph_search(problem):
    frontier = [(Node(problem.s_start))]  # Stack
    closed_list = set()
    log = []
    while frontier:
        node = frontier.pop()
        if debug:
          log.append((problem.state_str(node.state),node.solution(),len(frontier),problem.actions(node.state),problem.is_goal(node.state)))
        if problem.is_goal(node.state) or (debug and len(log)>100):
          return node.solution(), log  
        closed_list.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in closed_list and
                        child not in frontier)
    return None, log

def depth_limited_search(problem, limit=5):
    frontier = [(Node(problem.s_start))]  # Stack
    log = []
    while frontier:
        node = frontier.pop()
        if debug:
          log.append((problem.state_str(node.state),node.solution(),len(frontier),problem.actions(node.state),problem.is_goal(node.state)))
        if problem.is_goal(node.state):
          return node.solution(), log
        if node.depth<limit:
          frontier.extend(node.expand(problem))
    return None, log 

def iterative_deepening_search(problem):
  iterative_log = []
  for depth in range(1, sys.maxsize):
    result, log = depth_limited_search(problem, depth)
    iterative_log.extend(log)
    if result:
      return result, iterative_log
  return None, iterative_deepening_search

### Uniform Cost search

In [0]:

def best_first_graph_search(problem, f):
  node = Node(problem.s_start)
  frontier = PriorityQueue(f) #Priority Queue
  frontier.append(node)
  log = []
  closed_list = set()
  while frontier:
    if len(closed_list)%1000==0:
      print(f'size of closed list:{len(closed_list)}')
    node = frontier.pop()
    if debug:
      log.append((problem.state_str(node.state), node.solution(), node.path_cost,f(node),str(frontier), problem.actions(node.state), problem.is_goal(node.state)))    
    if problem.is_goal(node.state):
      return node.solution(), log
    closed_list.add(node.state)
    for child in node.expand(problem):
      if child.state not in closed_list and child not in frontier:
        frontier.append(child)
      elif child in frontier and f(child) < frontier[child]:
        del frontier[child]
        frontier.append(child)
  return None, log

#f = g: 
def uniform_cost_search(problem):
  def g(node):
    return node.path_cost
  return best_first_graph_search(problem, f=g)  

## Playing around

In [0]:
s_start = (
    3, 1, 2,
    4, 7, 5,
    6, 0, 8
)

easy_s_start = (
    1, 2, 0,
    3, 4, 5,
    6, 7, 8
)
tile_puzzle_problem = TilePuzzleProblem(s_start, 3)
tile_puzzle_easy_problem = TilePuzzleProblem(easy_s_start, 3)

routing_problem = RoutingProblem('Yavne','Bar-Ilan University',G)

### BFS

In [15]:
#solution, log = breadth_first_tree_search(tile_puzzle_problem)
solution, log = breadth_first_graph_search(tile_puzzle_problem)

#solution, log = breadth_first_tree_search(routing_problem)
#solution, log = breadth_first_graph_search(routing_problem)
print(f'Solution: {solution}')
format_search_log(log)

Solution: ['up', 'left', 'up']


,node to expand,path,left in frontier,valid actions,is_goal
0,"(3, 1, 2)(4, 7, 5)(6, 0, 8)",[],0,"[left, right, up]",False
1,"(3, 1, 2)(4, 7, 5)(0, 6, 8)",[left],2,"[right, up]",False
2,"(3, 1, 2)(4, 7, 5)(6, 8, 0)",[right],2,"[left, up]",False
3,"(3, 1, 2)(4, 0, 5)(6, 7, 8)",[up],2,"[left, right, up, down]",False
4,"(3, 1, 2)(0, 7, 5)(4, 6, 8)","[left, up]",4,"[right, up, down]",False
5,"(3, 1, 2)(4, 7, 0)(6, 8, 5)","[right, up]",5,"[left, up, down]",False
6,"(3, 1, 2)(0, 4, 5)(6, 7, 8)","[up, left]",6,"[right, up, down]",False
7,"(3, 1, 2)(4, 5, 0)(6, 7, 8)","[up, right]",7,"[left, up, down]",False
8,"(3, 0, 2)(4, 1, 5)(6, 7, 8)","[up, up]",8,"[left, right, down]",False
9,"(3, 1, 2)(7, 0, 5)(4, 6, 8)","[left, up, right]",9,"[left, right, up, down]",False


In [16]:
s_start = (
    3, 1, 2,
    0, 4, 5,
    6, 7, 8
)
problem = TilePuzzleProblem(s_start, 3)
solution, log = depth_first_graph_search(problem)
print(f'Solution: {solution}')
format_search_log(log)


Solution: ['down', 'right', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up']


,node to expand,path,left in frontier,valid actions,is_goal
0,"(3, 1, 2)(0, 4, 5)(6, 7, 8)",[],0,"[right, up, down]",False
1,"(3, 1, 2)(6, 4, 5)(0, 7, 8)",[down],2,"[right, up]",False
2,"(3, 1, 2)(6, 4, 5)(7, 0, 8)","[down, right]",2,"[left, right, up]",False
3,"(3, 1, 2)(6, 0, 5)(7, 4, 8)","[down, right, up]",3,"[left, right, up, down]",False
4,"(3, 0, 2)(6, 1, 5)(7, 4, 8)","[down, right, up, up]",5,"[left, right, down]",False
5,"(3, 2, 0)(6, 1, 5)(7, 4, 8)","[down, right, up, up, right]",6,"[left, down]",False
6,"(3, 2, 5)(6, 1, 0)(7, 4, 8)","[down, right, up, up, right, down]",6,"[left, up, down]",False
7,"(3, 2, 5)(6, 1, 8)(7, 4, 0)","[down, right, up, up, right, down, down]",7,"[left, up]",False
8,"(3, 2, 5)(6, 1, 8)(7, 0, 4)","[down, right, up, up, right, down, down, left]",7,"[left, right, up]",False
9,"(3, 2, 5)(6, 0, 8)(7, 1, 4)","[down, right, up, up, right, down, down, left,...",8,"[left, right, up, down]",False


### DFS

In [17]:
#solution, log = depth_first_tree_search(tile_puzzle_problem)
solution, log = depth_first_graph_search(tile_puzzle_problem)

#solution, log = depth_first_tree_search(routing_problem)
#solution, log = depth_first_graph_search(routing_problem)
print(f'Solution: {solution}')
format_search_log(log)

Solution: ['up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'right', 'up', 'up', 'left', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up', 'up', 'left']


,node to expand,path,left in frontier,valid actions,is_goal
0,"(3, 1, 2)(4, 7, 5)(6, 0, 8)",[],0,"[left, right, up]",False
1,"(3, 1, 2)(4, 0, 5)(6, 7, 8)",[up],2,"[left, right, up, down]",False
2,"(3, 0, 2)(4, 1, 5)(6, 7, 8)","[up, up]",4,"[left, right, down]",False
3,"(3, 2, 0)(4, 1, 5)(6, 7, 8)","[up, up, right]",5,"[left, down]",False
4,"(3, 2, 5)(4, 1, 0)(6, 7, 8)","[up, up, right, down]",5,"[left, up, down]",False
5,"(3, 2, 5)(4, 1, 8)(6, 7, 0)","[up, up, right, down, down]",6,"[left, up]",False
6,"(3, 2, 5)(4, 1, 8)(6, 0, 7)","[up, up, right, down, down, left]",6,"[left, right, up]",False
7,"(3, 2, 5)(4, 0, 8)(6, 1, 7)","[up, up, right, down, down, left, up]",7,"[left, right, up, down]",False
8,"(3, 0, 5)(4, 2, 8)(6, 1, 7)","[up, up, right, down, down, left, up, up]",9,"[left, right, down]",False
9,"(3, 5, 0)(4, 2, 8)(6, 1, 7)","[up, up, right, down, down, left, up, up, right]",10,"[left, down]",False


### Uniform Cost search

In [18]:
#solution, log = uniform_cost_search(tile_puzzle_problem)
solution, log = uniform_cost_search(routing_problem)
print(f'Solution: {solution}')
format_search_with_costs_log(log)

size of closed list:0
Solution: ['Aluf Sade Interchange', 'Bar-Ilan University']


,node to expand,path,g,f,left in frontier,valid actions,is_goal
0,Yavne,[],0,0,[],"(Ashdod, Gan-Yavne, Tel-Aviv, Aluf Sade Interc...",False
1,Rehovot,[Rehovot],6,6,"[(11, <Ashdod>), (13, <Gan-Yavne>), (25, <Tel-...","(Yavne, Aluf Sade Interchange)",False
2,Ashdod,[Ashdod],11,11,"[(13, <Gan-Yavne>), (26, <Aluf Sade Interchang...","(Ashkelon, Gan-Yavne, Yavne)",False
3,Gan-Yavne,"[Ashdod, Gan-Yavne]",12,12,"[(25, <Tel-Aviv>), (26, <Aluf Sade Interchange...","(Ashdod, Yavne)",False
4,Tel-Aviv,[Tel-Aviv],25,25,"[(26, <Aluf Sade Interchange>), (46, <Ashkelon>)]","(Yavne, Aluf Sade Interchange)",False
5,Aluf Sade Interchange,[Aluf Sade Interchange],26,26,"[(46, <Ashkelon>)]","(Yavne, Tel-Aviv, Rehovot, Bar-Ilan University)",False
6,Bar-Ilan University,"[Aluf Sade Interchange, Bar-Ilan University]",27,27,"[(46, <Ashkelon>)]","(Aluf Sade Interchange, Kiryat Ono)",True


In [19]:
routing_problem2 = RoutingProblem('S','G',G2)
solution, log = uniform_cost_search(routing_problem2)
print(f'Solution: {solution}')
format_search_with_costs_log(log)

size of closed list:0
Solution: ['A', 'E', 'F', 'G']


,node to expand,path,g,f,left in frontier,valid actions,is_goal
0,S,[],0,0,[],"(A, D)",False
1,D,[D],4,4,"[(5, <A>)]","(S, E)",False
2,A,[A],5,5,"[(7, <E>)]","(S, B, E)",False
3,E,"[A, E]",6,6,"[(9, <B>)]","(D, F, A, B)",False
4,B,"[A, B]",9,9,"[(10, <F>)]","(A, C, E)",False
5,F,"[A, E, F]",10,10,"[(13, <C>)]","(E, G)",False
6,C,"[A, B, C]",13,13,"[(13, <G>)]",(B),False
7,G,"[A, E, F, G]",13,13,[],(F),True


In [20]:
G2['S']

{'A': 5, 'D': 4}

# Informed Search algorithms

## Implementation

In [0]:
def greedy_best_first_graph_search(problem, h):
  return best_first_graph_search(problem, f=h)

def astar_search(problem, h):
  def g(node):
    return node.path_cost
  return best_first_graph_search(problem, f=lambda n: g(n)+h(n))

## Playing around

In [0]:
#h1: number of misplaced tiles (note that blank is not counted)
class TilePuzzleMisplacedTiles:
  def __init__(self, n):
    self.goal = tuple(range(0,n**2))
  def h(self,node):
    return sum(s != g for (s, g) in zip(node.state, self.goal))- int(node.state[0]!=0)

#h2: sum of manhattan distances of all tiles (note that blank is not counted)
class TilePuzzleManhattanDistance:
  def __init__(self, n):
    self.goal = tuple(range(1,n**2)) #note that blank is omitted
  def h(self,node):
    res = 0
    for i in self.goal:
      x,y = i//3 , i%3
      nx,ny = node.state.index(i)//3 , node.state.index(i)%3
      res+= abs(x-nx) + abs(y-ny)
    return res

In [23]:
solution, log = astar_search(tile_puzzle_problem, h=TilePuzzleMisplacedTiles(3).h)
print(f'Solution: {solution}')
format_search_with_costs_log(log)

size of closed list:0
Solution: ['up', 'left', 'up']


,node to expand,path,g,f,left in frontier,valid actions,is_goal
0,"(3, 1, 2)(4, 7, 5)(6, 0, 8)",[],0,3,[],"[left, right, up]",False
1,"(3, 1, 2)(4, 0, 5)(6, 7, 8)",[up],1,3,"[(5, <(3, 1, 2, 4, 7, 5, 0, 6, 8)>), (5, <(3, ...","[left, right, up, down]",False
2,"(3, 1, 2)(0, 4, 5)(6, 7, 8)","[up, left]",2,3,"[(5, <(3, 0, 2, 4, 1, 5, 6, 7, 8)>), (5, <(3, ...","[right, up, down]",False
3,"(0, 1, 2)(3, 4, 5)(6, 7, 8)","[up, left, up]",3,3,"[(5, <(3, 0, 2, 4, 1, 5, 6, 7, 8)>), (5, <(3, ...","[right, down]",True


In [24]:
hard_s_start = (7, 2, 4,
                5, 0, 6,
                8, 3, 1)

h1 = TilePuzzleMisplacedTiles(3).h
h2 = TilePuzzleManhattanDistance(3).h
n = Node(hard_s_start)
print(f'Number of misplaced tiles: {h1(n)}. Manhattan Distance: {h2(n)}.')

Number of misplaced tiles: 8. Manhattan Distance: 18.


In [25]:
hard_tile_puzzle_problem = TilePuzzleProblem(hard_s_start, 3)

#solution, log = astar_search(hard_tile_puzzle_problem, h=h1) #<= this will take a while to run
%time solution, log = astar_search(hard_tile_puzzle_problem, h=h2)
print(f'Solution of length {len(solution)}: {solution}')
format_search_with_costs_log(log)

size of closed list:0
size of closed list:1000
size of closed list:2000
CPU times: user 2.51 s, sys: 66.8 ms, total: 2.58 s
Wall time: 2.59 s
Solution of length 26: ['left', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left']


,node to expand,path,g,f,left in frontier,valid actions,is_goal
0,"(7, 2, 4)(5, 0, 6)(8, 3, 1)",[],0,18,[],"[left, right, up, down]",False
1,"(7, 2, 4)(0, 5, 6)(8, 3, 1)",[left],1,18,"[(18, <(7, 2, 4, 5, 3, 6, 8, 0, 1)>), (18, <(7...","[right, up, down]",False
2,"(0, 2, 4)(7, 5, 6)(8, 3, 1)","[left, up]",2,18,"[(18, <(7, 2, 4, 5, 3, 6, 8, 0, 1)>), (18, <(7...","[right, down]",False
3,"(7, 2, 4)(5, 3, 6)(8, 0, 1)",[down],1,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[left, right, up]",False
4,"(7, 2, 4)(5, 3, 6)(0, 8, 1)","[down, left]",2,18,"[(18, <(7, 2, 4, 5, 3, 6, 8, 1, 0)>), (18, <(7...","[right, up]",False
5,"(7, 2, 4)(5, 3, 6)(8, 1, 0)","[down, right]",2,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[left, up]",False
6,"(7, 2, 4)(5, 3, 0)(8, 1, 6)","[down, right, up]",3,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[left, up, down]",False
7,"(7, 2, 0)(5, 3, 4)(8, 1, 6)","[down, right, up, up]",4,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[left, down]",False
8,"(7, 0, 2)(5, 3, 4)(8, 1, 6)","[down, right, up, up, left]",5,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[left, right, down]",False
9,"(0, 7, 2)(5, 3, 4)(8, 1, 6)","[down, right, up, up, left, left]",6,18,"[(18, <(7, 2, 4, 5, 6, 0, 8, 3, 1)>), (20, <(2...","[right, down]",False
